In [1]:
import datetime

import numpy as np
import shapely
from czml3 import CZML_VERSION, Document, Packet
from czml3.properties import (
    Clock,
    Color,
    Material,
    Polygon,
    PositionList,
    SolidColorMaterial,
)
from czml3.types import IntervalValue, TimeIntervalCollection

import czml3_ext
from czml3_ext import packets
from czml3_ext.colours import RGBA_blue, RGBA_orange, RGBA_white, create_palette
from czml3_ext.helpers import get_border

In [ ]:
print(f"Using czml3-ext version: {czml3_ext.__version__}")

## Time
czml3-ext works fully with czml3.

In [3]:
border = packets.border("Israel")

In [4]:
ddm_LLA_israel = get_border("israel")
poly_shape = shapely.Polygon(ddm_LLA_israel[:, [0, 1], 0].tolist())
x = np.linspace(ddm_LLA_israel[:, 1, 0].min(), ddm_LLA_israel[:, 1, 0].max(), 20)
y = np.linspace(ddm_LLA_israel[:, 0, 0].min(), ddm_LLA_israel[:, 0, 0].max(), 20)
xv, yv = np.meshgrid(x, y)
ddm_points = []
for i in range(x.shape[0]):
    for j in range(y.shape[0]):
        if not poly_shape.contains(shapely.Point(yv[j, i], xv[j, i])):
            continue
        ddm_points.append([yv[j, i], xv[j, i], 0])
ddm_LLA_points = np.array(ddm_points).reshape((-1, 3, 1))

In [5]:
rgba = create_palette(
    [RGBA_blue, RGBA_white, RGBA_orange, RGBA_blue], ddm_LLA_points.shape[0]
)

In [6]:
start = datetime.datetime.now()
s_step = 20
cc = []
for i in range(len(rgba)):
    d = []
    for ii, c in enumerate(rgba[i:] + rgba[:i]):
        d.append(
            IntervalValue(
                start=start + datetime.timedelta(seconds=s_step) * ii,
                end=start + datetime.timedelta(seconds=s_step) * (ii + 1),
                value=Color(rgba=c),
            )
        )
    cc.append(d)

In [7]:
grid = packets.grid(
    ddm_LLA_points,
    polygon=[
        Polygon(
            positions=PositionList(cartographicDegrees=[0, 0, 0]),
            material=Material(
                solidColor=SolidColorMaterial(color=TimeIntervalCollection(values=c))
            ),
            outlineColor=Color(rgba=[255, 255, 255, 255]),
            outline=True,
        )
        for c in cc
    ],
    name=[f"Square {i}" for i in range(len(rgba))],
    description=[
        f"Colour: [{c[0]:.2f}, {c[1]:.2f}, {c[2]:.2f}, {c[3]:.2f}]" for c in rgba
    ],
    ddm_LLA_cut=ddm_LLA_israel,
)

In [8]:
doc = Document(
    packets=(
        [
            Packet(
                name="simple",
                id="document",
                version=CZML_VERSION,
                clock=IntervalValue(
                    start=start,
                    end=start + datetime.timedelta(seconds=s_step) * (ii + 1),
                    value=Clock(currentTime=start, multiplier=1200),
                ),
            )
        ]
        + border
        + grid
    )
)

In [9]:
with open("example_time.czml", "w") as f:
    f.write(doc.dumps())